Entrar no venv `.\tcc\Scripts\activate.bat`. Download Opencv em [Source Forge](https://sourceforge.net/projects/opencvlibrary/files/opencv-win/3.4.1/opencv-3.4.1-vc14_vc15.exe/download) e no caminho `build/x64/vc15/bin` possui os executáveis necessários.

Com os executáveis, agora basta configurar o Dataset. Imagens negativas obtidas (8000 imagens) do Imagenet e imagens positivas que são só o corpo dos resistores. As imagens negativas são renomeadas e listadas em um txt com o seguinte script `renameNegatives.py`:

Devido a erros de ROI na geração das imagens (garantir mesma resolução), é necessário executar o script `removeImagesDiffRes.py` que remove imagens que não estão no padrão de 224x224. Isso resultou na exclusão de 8000-6217 = 1783 imagens. Com isso, podemos executar o `opencv_createsamples.exe` passando as informações necessárias, resultando na geração de 1800 imagens positivas. Para cada imagem de resistor deve ser executado uma vez e entre isso o script `shuffleNegativeTxt`:

In [ ]:
execs_opencv\opencv_createsamples -img data_set/positives/resistor7.jpg -bg data_set/negatives/negatives.txt -info data_set/positives/positive7/positives7.lst -maxxangle 0.5 -maxyangle 0.5 -maxzangle 0.5 -num 1800 -bgcolor 255 -bgthresh 8

Para cada um criado tem que ser colocado as imagens e o .lst na sua pasta com numeração. E também o .vec deve ser criado para cada um:

In [ ]:
execs_opencv\opencv_createsamples -info data_set/positives/positive7/positives7.lst -num 1800 -w 40 -h 15 -vec data_set/positives/vec/positives7.vec

Colcoar todos os .vec em uma pasta vec e deve-se unir os .vec gerados usando o script que vai gerar um vector.vec

In [ ]:
python scripts/mergevec.py -v data_set\positives\vec -o data_set\positives\vec\vetor.vec

Execução dentro da pasta negatives, com o opencv_traincascade.exe e os dll dentro tambem. Além disso, a pasta do -data (aqui chamada de classifier) deve ser criada antes.

Primeiro o v1, depois o com dois resistores.

In [ ]:
opencv_traincascade -data C:\Users\nana_\Desktop\2025.2\TCC-2\classifier -vec C:\Users\nana_\Desktop\2025.2\TCC-2\data_set\positives\positives.vec -bg negatives.txt -numPos 1300 -numNeg 800 -numStages 10 -w 40 -h 15 -precalcBufSize 4096 -precalcIdxBufSize 4096

In [ ]:
opencv_traincascade -data C:\Users\nana_\Desktop\2025.2\TCC-2\classifier -vec C:\Users\nana_\Desktop\2025.2\TCC-2\data_set\positives\vec\vetor.vec -bg negatives.txt -numPos 1300 -numNeg 800 -numStages 10 -w 40 -h 15 -precalcBufSize 4096 -precalcIdxBufSize 4096

In [ ]:
opencv_traincascade -data C:\Users\nana_\Desktop\2025.2\TCC-2\classifier -vec C:\Users\nana_\Desktop\2025.2\TCC-2\data_set\positives\vec\vetor.vec -bg negatives.txt -numPos 5000 -numNeg 6000 -numStages 10 -w 40 -h 15 -precalcBufSize 4096 -precalcIdxBufSize 4096

Anotação


In [ ]:
execs_opencv\opencv_annotation --annotations=data_set_fotos_resize\pos.txt --images=data_set_fotos_resize\

No pos.txt criado é necessário eliminar o caminho de cada imagem, e deixar somente o nome do arquivo. E também elimitar aquelas imagnes que não possuem nada marcado, as com 0. O -num basta ser maior que o número de anotações criadas.

In [ ]:
execs_opencv\opencv_createsamples -info data_set_fotos_resize\pos.txt -w 40 -h 15 -num 400 -vec data_set_fotos_resize\pos.vec

Executar dentro da pasta data_set negatives, para aproveitas os negativos.

In [ ]:
opencv_traincascade -data C:\Users\nana_\Documents\2025.2\TCC-2\cascade_fotos -vec C:\Users\nana_\Documents\2025.2\TCC-2\data_set_fotos_resize\pos.vec -bg negatives.txt -numPos 30 -numNeg 100 -numStages 10 -w 40 -h 15 -precalcBufSize 4096 -precalcIdxBufSize 4096